In [2]:
!pip3 install pykeen

  Using cached pykeen-1.10.1-py3-none-any.whl (739 kB)
  Using cached dataclasses_json-0.6.3-py3-none-any.whl.metadata (25 kB)
  Using cached click_default_group-1.2.4-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached optuna-3.5.0-py3-none-any.whl.metadata (17 kB)
  Using cached more_click-0.1.2-py3-none-any.whl (6.7 kB)
  Using cached more_itertools-10.2.0-py3-none-any.whl.metadata (34 kB)
  Using cached pystow-0.5.2-py3-none-any.whl.metadata (8.9 kB)
  Using cached docdata-0.0.3-py3-none-any.whl (5.8 kB)
  Using cached class_resolver-0.4.2-py3-none-any.whl (24 kB)
  Using cached rexmex-0.1.3-py3-none-any.whl
  Using cached torch_max_mem-0.1.3-py3-none-any.whl.metadata (7.4 kB)
  Using cached torch_ppr-0.0.8-py3-none-any.whl (12 kB)
  Using cached alembic-1.13.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached colorlog-6.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached SQLAlchemy-2.0.25-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached 

In [5]:
import pykeen
from pykeen.datasets import Nations
from pykeen.pipeline import pipeline
from pykeen.predict import predict_triples
import torch

In [8]:
dataset = Nations()

training_data = dataset.training.mapped_triples

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


pipeline_results = pipeline(
    model="TransE",  # TransE model for relation prediction
    dataset=dataset,  # Nations dataset
    training_kwargs=dict(num_epochs=100),  # Number of training epochs
    device=device
)

print(pipeline_results)

INFO:pykeen.pipeline.api:Using device: cuda


Training epochs on cuda:0: 100%|██████████| 100/100 [00:08<00:00, 12.22epoch/s, loss=0.769, prev_loss=0.737]
INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=201.
Evaluating on cuda:0: 100%|██████████| 201/201 [00:00<00:00, 16.7ktriple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 0.02s seconds


PipelineResult(random_seed=4062164729, model=TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(14, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(55, 50)
    )
  )
  (weight_regularizers): ModuleList()
), training=TriplesFactory(num_entities=14, num_relations=55, create_inverse_triples=False, num_triples=1592, path="/home/space/miniconda3/envs/know/lib/python3.8/site-packages/pykeen/datasets/nations/train.txt"), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x7f64748d3970>, losses=[1.351236070905413, 1.4376486369541712, 1.3369601283754622, 1.306329607963562, 1.3350036484854562, 1.2912931782858712, 1.1927975927080428, 1.2286902495792933, 1.222324218068804, 1.2023182681628637, 1.2248401301247733, 1.2228398493358068, 1.2016646010535104, 1.2131702218736922, 1.1590

In [9]:
pack = predict_triples(model=pipeline_results.model, triples=dataset.validation)
df = pack.process(factory=pipeline_results.training).df

df.nlargest(n=10, columns="score")

,head_id,head_label,relation_id,relation_label,tail_id,tail_label,score
175,12,usa,47,tourism,4,egypt,-7.054984
90,7,israel,13,eemigrants,12,usa,-7.078137
159,11,uk,40,relstudents,12,usa,-7.179070
176,12,usa,47,tourism,11,uk,-7.525523
35,2,china,26,negativecomm,12,usa,-7.539259
57,3,cuba,26,negativecomm,12,usa,-7.689585
170,12,usa,31,pprotests,10,poland,-7.815037
65,4,egypt,18,exports3,11,uk,-7.880906
139,10,poland,48,tourism3,13,ussr,-8.030716
93,7,israel,24,militaryalliance,12,usa,-8.072074
